In [ ]:
# Load the uploaded CSV file to inspect the data structure and identify potential issues

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'monatszahlen2405_verkehrsunfaelle_export_31_05_24_r.csv'
data = pd.read_csv(file_path)

# Display basic information and the first few rows of the data
data_info = data.info()
data_head = data.head()
data_unique_months = data['MONAT'].unique()  # Check unique values in 'MONAT' column

data_info, data_head, data_unique_months


In [ ]:
# Extracting year and month correctly from 'MONAT' and creating a new 'Date' column
data['Year'] = data['MONAT'].str[:4]  # Extract year from the first four characters
data['Month'] = data['MONAT'].str[4:]  # Extract month from the remaining characters

# Convert 'Year' and 'Month' to datetime
data['Date'] = pd.to_datetime(data['Year'] + '-' + data['Month'], format='%Y-%m', errors='coerce')

# Check for any invalid dates
invalid_dates = data[data['Date'].isna()]  # Identify any rows that couldn't be converted

# Display head of the corrected data and invalid dates if any
data_corrected_head = data.head()
invalid_dates_count = invalid_dates.shape[0]

data_corrected_head, invalid_dates_count, invalid_dates.head()  # Display results


In [ ]:
# Filter out rows with invalid 'Date' values (those where conversion to datetime failed)
cleaned_data = data[data['Date'].notna()]

# Verify the data cleaning step
cleaned_data_head = cleaned_data.head()
cleaned_data_summary = cleaned_data.describe()

# Display the cleaned data and summary statistics
cleaned_data_head, cleaned_data_summary
